In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Importing the required libraries
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
# Load configuration
base_dir = os.getcwd()
dataset_dir = os.path.join(base_dir, 'dataset')
model_path_dir = os.path.join(base_dir, 'saved_model')

In [ ]:
# Directories
train_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'valid')
test_dir = os.path.join(dataset_dir, 'test')

# Image dimensions
image_height = 128
image_width = 128
batch_size = 64

# Data generators with augmentation for training data
train_image_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
validation_image_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_image_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data_gen = train_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    shuffle=True,
    target_size=(image_height, image_width),
    class_mode='categorical'
)
val_data_gen = validation_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=validation_dir,
    target_size=(image_height, image_width),
    class_mode='categorical'
)
test_data_gen = test_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=test_dir,
    target_size=(image_height, image_width),
    class_mode='categorical'
)

In [ ]:
# Define the input shape explicitly
input_shape = (image_height, image_width, 3)

# Loading the MobileNetV2 model pre-trained on ImageNet
base_model = MobileNetV2(input_shape=input_shape,
                         include_top=False,
                         weights='imagenet')

# Using the entire MobileNetV2 model as the base model
base_model.trainable = True

# Building the model with an explicit Input layer
model = Sequential([
    Input(shape=input_shape),  # Explicit Input layer
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Recompiling the model with a lower learning rate for fine tuning
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    verbose = 1,
    restore_best_weights=True,
    patience=18,
    mode='max'
)

In [ ]:
# Model training
history = model.fit(
    train_data_gen,
    validation_data=val_data_gen,
    epochs=30,
    callbacks=[early_stopping] # type: ignore
)

In [ ]:
# Evaluating the model
test_loss, test_acc = model.evaluate(test_data_gen)
print('Test accuracy:', test_acc)

In [ ]:
# Saving the model
model.save('test.keras')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
loaded_model = load_model('test.keras')